# Review

- 웹수집 단계 
    - 데이타 요청
    - 문자열 데이타 (html 페이지) 
    - 파싱(데이타 추출) => 문자열 
    - 테이블 구조 데이타 ( 2차원 리스트, 리스트 딕셔너리 ) 
    - 텍스트 파일
    - csv 파일
        - 판다스의 데이타프레임 구조 
        
## 데이타 요청        
    - requests 
    - urllib 

## 파싱 
    - 뷰티풀숩
    - 셀레니움 

## 뷰티풀숩
    - 도트 방식으로 찾기 
    - find(), find_all()
    - select_one(), select()
    - text, name, attrs

# 퀴즈 
- 슬라이드 94
- https://www.gogung.go.kr/gogung/pgm/psgudMng/appointList.do?gubunCd=PSGUD_006&menuNo=800067
- 지정문화재 페이지에서 제목, 상세설명, 이미지URL 정보 추출 
- 텍스트 파일로 최종 저장 

In [1]:
# 0) 모듈 임포트 

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 

url = 'https://www.gogung.go.kr/gogung/pgm/psgudMng/appointList.do?gubunCd=PSGUD_006&menuNo=800067'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [9]:
# 2) 파싱 
# target = soup.select('.board-gallery.type2 li a')
target_list = soup.select('.board-gallery.type2 > li > a')
print(len(target_list))

print(target_list[-1])

5
<a href="appointView.do?psgudSn=358107&amp;menuNo=800067&amp;gubunCd=PSGUD_006" title="백자달항아리 페이지 이동">
<span class="img">
<img alt="백자달항아리" src="/cmmn/file/imageSrc.do?atchFileId=00006726JP&amp;fileSn=1&amp;thumb=Y"/>
</span>
<span class="subject ellipsis-line2">백자달항아리</span>
<span class="text ellipsis-line3">조선시대에 제작한 백자이다. 
둥글고 유백색(乳白色)의 형태가 둥근 달을 연상하게 되어 ‘달항아리’라고 부른다. 이 백자는 완전한 좌우대칭은 아니지만 약간 비틀어지고 변...</span>
</a>


In [20]:
# 첫번째 태그에서 데이타 추출 
# 제목, 설명, 이미지URL
print('제목 :', target_list[0].select_one('.subject').text)
print('제목 :', target_list[0].select_one('img')['alt'])
print('설명 :', target_list[0].select_one('.text').text)
base_url = 'https://www.gogung.go.kr'
print('이미지 URL :', base_url+target_list[0].select_one('img')['src'])
print('-'*20)
print(target_list[0])

제목 : 창경궁 자격루 항아리
제목 : 창경궁 자격루 항아리
설명 : ‘스스로 치는 시계’라는 뜻의 자격루는 물의 증가나 감소에 따라 자동으로 시각을 알려주는 첨단 물시계로, 조선 시대의 국가 표준시계였다. 자격루 누기는 자격루를 구성하는 장치들 중...
이미지 URL : https://www.gogung.go.kr/cmmn/file/imageSrc.do?atchFileId=00022463TB&fileSn=1&thumb=Y
--------------------
<a href="appointView.do?psgudSn=370348&amp;menuNo=800067&amp;gubunCd=PSGUD_006" title="창경궁 자격루 항아리 페이지 이동">
<span class="img">
<img alt="창경궁 자격루 항아리" src="/cmmn/file/imageSrc.do?atchFileId=00022463TB&amp;fileSn=1&amp;thumb=Y"/>
</span>
<span class="subject ellipsis-line2">창경궁 자격루 항아리</span>
<span class="text ellipsis-line3">‘스스로 치는 시계’라는 뜻의 자격루는 물의 증가나 감소에 따라 자동으로 시각을 알려주는 첨단 물시계로, 조선 시대의 국가 표준시계였다. 자격루 누기는 자격루를 구성하는 장치들 중...</span>
</a>


In [95]:
# 3) 테이블 구조 데이타 ( 2차원 리스트 )
result_list = []
for tag in target_list:
    title = tag.select_one('.subject').text
    # 글문단에서 개행및 여백 정리 
    # csv 파일로 저장할 때 에러 해결을 위해 
    # .replace('\xa0',' ') 추가 필요  
    contents = tag.select_one('.text').text.replace('  ','').strip().replace('\n',' ').replace('\xa0',' ')
#     contents = tag.select_one('.text').text.replace('  ','').strip().replace('\n',' ')
    img_url = base_url+tag.select_one('img')['src']
    # 리스트 생성후 데이타 삽입
    result_list.append([title, contents, img_url])

In [96]:
for title, contents, img_url in result_list:
    print(f'제목 : {title}')
    print(f'내용 : {contents}')
    print(f'이미지URL : {img_url}')
    print('='*30)

제목 : 창경궁 자격루 항아리
내용 : ‘스스로 치는 시계’라는 뜻의 자격루는 물의 증가나 감소에 따라 자동으로 시각을 알려주는 첨단 물시계로, 조선 시대의 국가 표준시계였다. 자격루 누기는 자격루를 구성하는 장치들 중...
이미지URL : https://www.gogung.go.kr/cmmn/file/imageSrc.do?atchFileId=00022463TB&fileSn=1&thumb=Y
제목 : 조선왕조실록 오대산 사고본
내용 : 『조선왕조실록(朝鮮王朝實錄)』은 조선 태조에서부터 조선 철종 때까지 25대 472년간(1392∼1863)의 역사를 일어난 순서대로 기록한 책이다. 이 중 오대산사고본(五臺山史庫本)...
이미지URL : https://www.gogung.go.kr/cmmn/file/imageSrc.do?atchFileId=00018144HF&fileSn=1&thumb=Y
제목 : 창덕궁 이문원 측우대
  원래는 측우기와 측우대는 창덕궁 내 이문원...
이미지URL : https://www.gogung.go.kr/cmmn/file/imageSrc.do?atchFileId=00006752XQ&fileSn=1&thumb=Y
제목 : 천상열차분야지도 각석
 이 지도의 내용은 두 부분으로 구성...
이미지URL : https://www.gogung.go.kr/cmmn/file/imageSrc.do?atchFileId=00006730AC&fileSn=1&thumb=Y
제목 : 백자달항아리
 둥글고 유백색(乳白色)의 형태가 둥근 달을 연상하게 되어 ‘달항아리’라고 부른다. 이 백자는 완전한 좌우대칭은 아니지만 약간 비틀어지고 변...
이미지URL : https://www.gogung.go.kr/cmmn/file/imageSrc.do?atchFileId=00006726JP&fileSn=1&thumb=Y


In [97]:
# 4) 텍스트파일로 저장하기 
with open('output/지정문화재.txt','w', encoding='utf-8') as f:
    for title, contents, img_url in result_list:
        f.write(f'제목 : {title}\n')
        f.write(f'내용 : {contents}\n')
        f.write(f'이미지URL : {img_url}\n')
        f.write('='*30)
        f.write('\n')

In [98]:
# 5) 판다스의 데이타프레임 구조로 변경하기 
df = pd.DataFrame(result_list, 
                  columns=['제목', '내용', '이미지 URL'])
df

,제목,내용,이미지 URL
0,창경궁 자격루 항아리,‘스스로 치는 시계’라는 뜻의 자격루는 물의 증가나 감소에 따라 자동으로 시각을 알...,https://www.gogung.go.kr/cmmn/file/imageSrc.do...
1,조선왕조실록 오대산 사고본,『조선왕조실록(朝鮮王朝實錄)』은 조선 태조에서부터 조선 철종 때까지 25대 472년...,https://www.gogung.go.kr/cmmn/file/imageSrc.do...
2,창덕궁 이문원 측우대,조선시대 강우량을 측정하는 측우기를 올려 놓았던 대석(臺石)이다. \r 원래는 측...,https://www.gogung.go.kr/cmmn/file/imageSrc.do...
3,천상열차분야지도 각석,천체의 형상을 직육면체의 돌에 새겨 놓은 지도이다. \r 이 지도의 내용은 두 부분...,https://www.gogung.go.kr/cmmn/file/imageSrc.do...
4,백자달항아리,조선시대에 제작한 백자이다. \r 둥글고 유백색(乳白色)의 형태가 둥근 달을 연상하...,https://www.gogung.go.kr/cmmn/file/imageSrc.do...


In [99]:
# 6) csv 파일로 저장하기 
# index=False 행인덱스 제거해서 저장 
# df.to_csv('output/지정문화재.csv', index=False)
df.to_csv('output/지정문화재.csv', index=False, encoding='cp949')

In [101]:
# ls output

# W3schools의 color  

- 영문색상명, 16진수색상코드값, 페이지URL
- https://www.w3schools.com/colors/colors_names.asp
- 최종 결과물 => csv

In [104]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 

url = 'https://www.w3schools.com/colors/colors_names.asp'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
# print(soup)

In [117]:
# 2) 파싱 
# 소스 파악 
# 클래스명 확인 innerbox
target_list = soup.find_all(class_='innerbox')
# target_list = soup.select('.innerbox')
print(len(target_list))
# print(target_list[0])
print()
# print(target_list[-1])

148



In [115]:
# 개별 데이타 추출 테스트 
# 영문색상명, 16진수색상코드값, 페이지URL
base_url ='https://www.w3schools.com'
print('영문색상명 =>', target_list[0].a.text)
print('16진수색상코드값 =>', 
      target_list[0].find(class_='colorhexspan').text)
print('페이지URL =>', 
      base_url+target_list[0].a['href'])

영문색상명 => AliceBlue
16진수색상코드값 => #F0F8FF
페이지URL => https://www.w3schools.com/colors/color_tryit.asp?color=AliceBlue


In [118]:
# 3) 테이블 구조 데이타 ( 2차원 리스트 )
result_list = []
for tag in target_list:
    color_name = tag.a.text
    hexcode = tag.find(class_='colorhexspan').text
    url = base_url + tag.a['href']
    result_list.append([color_name, hexcode, url])

In [120]:
# for color_name, hexcode, url in result_list:
#     print(f'영문색상명 : {color_name}')
#     print(f'16진수 : {hexcode}')
#     print(f'URL : {url}')
#     print('='*30)

In [126]:
# 4) 판다스의 데이타프레임 구조로 변경한 후 csv 파일로 저장하기 
df = pd.DataFrame(result_list, 
                  columns=['색상명', '16진수', 'URL'])
# df # 전체보기
# df.head() # 앞에 5 행만 
# df.tail() # 마지막 5개만 

In [124]:
df.to_csv('output/웹색상.csv', index=False, encoding='cp949')

In [125]:
ls output\*.csv

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F2CA-487A

 c:\pyclass\output 디렉터리

2023-02-08  오후 04:54               579 블로터랭킹.csv
2023-02-09  오전 11:30            12,437 웹색상.csv
2023-02-09  오전 10:55             1,287 지정문화재.csv
               3개 파일              14,303 바이트
               0개 디렉터리  928,626,999,296 바이트 남음


# 퀴즈 - 슬라이드 101


## Yes24 베스트셀러 정보 저장하기

- http://www.yes24.com/Main/default.aspx
- 국내 도서 1~10 위의 데이타를 CSV 파일로 저장하여라
- 추출정보 : 순위, 책제목, 저자, 출판사, 표지URL, URL

In [128]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 

url = 'http://www.yes24.com/Main/default.aspx'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
# print(soup)

In [133]:
# 2) 파싱 
# 클래스명 확인 tp02
target_list = soup.find_all(class_='tp02')
print(len(target_list))
# print(target_list[0])
# print()
# print(target_list[-1])

10


In [159]:
# 데이타 추출
# 순위, 책제목, 저자, 출판사, 표지URL, URL
print('순위 => ', target_list[-1].strong.text)
# img 태그의 alt 속성값 
print('책제목 => ', target_list[-1].img['alt'])
# 2개의 em 태그중 2번째의 텍스트로 추출
print('저자 => ', target_list[-1].select('em')[1].text)
# title 속성값에서 마지막 단어 
# split() => 공백을 기준으로 리스트로 생성 
# print('출판사 => ', target_list[0].a['title'])
print('출판사 => ', target_list[-1].a['title'].split()[-1])
print('표지URL => ', target_list[-1].img['src'])
base_url = 'http://www.yes24.com'
print('상세 페이지 URL => ', base_url+target_list[-1].a['href'])

순위 =>  10위
책제목 =>  슬램덩크 신장재편판 19
저자 =>  이노우에 타케히코 글,그림
출판사 =>  대원
표지URL =>  https://image.yes24.com/Goods/66360835/S
상세 페이지 URL =>  http://www.yes24.com/Product/Goods/66360835


In [162]:
# 3) 테이블 구조 데이타 ( 2차원 리스트 )
result_list = []
for tag in target_list:
    rank = tag.strong.text
    title = tag.img['alt']
    writer = tag.select('em')[1].text
    pub = tag.a['title'].split()[-1]
    book_img = tag.img['src']
    link = base_url+tag.a['href']
    result_list.append([rank,title,writer, pub, book_img, link])

In [164]:
result_list[0]

['1위',
 '만일 내가 인생을 다시 산다면 (10만 부 기념 스페셜 에디션)',
 '김혜남 저',
 '메이븐',
 'https://image.yes24.com/Goods/115142458/S',
 'http://www.yes24.com/Product/Goods/115142458']

In [170]:
# 4) 판다스의 데이타프레임 구조로 변경한 후 csv 파일로 저장하기 
df = pd.DataFrame(result_list, 
         columns=['순위','책제목','저자','출판사','표지','링크'])
# df # 전체보기
# df.head(1) # 앞에 1행만 
# df.tail(1) # 마지막 1개만 

In [169]:
df.to_csv('output/예스24랭킹.csv', index=False, encoding='cp949')

# 퀴즈 - 슬라이드 103


## 네이버 요일별 웹툰 저장하기

- https://comic.naver.com/webtoon/weekday
- 요일별 전체 웹툰 영역의 웹툰 정보를 CSV 파일로 저장하여라
- 요일 / 웹툰제목 / 관련 URL

In [174]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 

url = 'https://comic.naver.com/webtoon/weekday'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
# print(soup)

In [177]:
# 2) 파싱 
# target_list = soup.find_all(class_='thumb')
target_list = soup.select('.thumb')
print(len(target_list))
print(target_list[0])
print()
print(target_list[-1])

574
<div class="thumb">
<a href="/webtoon/list?titleId=648419&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.img','','1')">
<img alt="뷰티풀 군바리" height="107" onerror="this.src='https://ssl.pstatic.net/static/comic/images/migration/common/blank.gif'" src="https://shared-comic.pstatic.net/thumb/webtoon/648419/thumbnail/thumbnail_IMAG21_d9398229-cbfd-47dc-9208-0a6fb936f3a7.jpg" title="뷰티풀 군바리" width="83"/><span class="mask"></span>
</a>
</div>

<div class="thumb">
<a href="/webtoon/list?titleId=801428&amp;weekday=sun" onclick="nclk_v2(event,'thm*S.img','','85')">
<img alt="천년간 노려왔습니다" height="107" onerror="this.src='https://ssl.pstatic.net/static/comic/images/migration/common/blank.gif'" src="https://shared-comic.pstatic.net/thumb/webtoon/801428/thumbnail/thumbnail_IMAG21_50e8a89e-b0b6-44ad-8a98-0850328943b4.jpg" title="천년간 노려왔습니다" width="83"/><span class="mask"></span>
</a>
</div>


In [ ]:
<div class="thumb">
	<a href="/webtoon/list?titleId=774863&amp;weekday=mon" 
       onclick="nclk_v2(event,'thm*m.img','','5')">
		<img onerror="this.src='https://ssl.pstatic.net/static/comic/images/migration/common/blank.gif'" 
           src="https://shared-comic.pstatic.net/thumb/webtoon/774863/thumbnail/thumbnail_IMAG21_3689684179498578529.jpg" 
            width="83" height="107" 
            title="팔이피플" 
            alt="팔이피플">
        <span class="mask"></span>
	</a>
</div>

In [206]:
# 데이타 추출
# 요일 / 웹툰제목 / 관련 URL
base_url = 'https://comic.naver.com'
week_list1 = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
week_list2 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
print('요일 => ', target_list[0].a['href'][-3:])
# = 를 기준으로 분리 => 리스트
# print('요일 => ', target_list[0].a['href'])
# print('요일 => ', target_list[0].a['href'].split('=')[-1])
# 리스트명.index(데이타값) => 데이타값에 해당하는 인덱스값 추출 
print('요일값 = ', target_list[0].a['href'][-3:])
idx = week_list1.index(target_list[0].a['href'][-3:])
print('인덱스 추출하기 = ', idx)
print('해당 요일은? ', week_list2[idx])
print()
print('웹툰제목 = ', target_list[0].img['alt'])
print('URL = ', base_url+target_list[0].a['href'])

요일 =>  mon
요일값 =  mon
인덱스 추출하기 =  0
해당 요일은?  월요일

웹툰제목 =  뷰티풀 군바리
URL =  https://comic.naver.com/webtoon/list?titleId=648419&weekday=mon


In [207]:
# 3) 테이블 구조 데이타 ( 2차원 리스트 )
result_list = []
base_url = 'https://comic.naver.com'
week_list1 = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
week_list2 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
for tag in target_list:
    # 요일값 추출하기 
    temp = tag.a['href'][-3:]
    idx = week_list1.index(temp)
    week = week_list2[idx]
    # 웹툰제목 
    title = tag.img['alt']
    # url 상세 주소
    url = base_url+tag.a['href']
    result_list.append([week, title, url])

In [209]:
# 4) 판다스의 데이타프레임 구조로 변경한 후 csv 파일로 저장하기 
df = pd.DataFrame(result_list, 
         columns=['요일','웹툰제목','링크'])
# df # 전체보기
# 랜덤하게 샘플 뽑아서 출력
df.sample(5)

,요일,웹툰제목,링크
562,일요일,하렘에서 살아남기,https://comic.naver.com/webtoon/list?titleId=7...
136,화요일,내남친 킹카만들기,https://comic.naver.com/webtoon/list?titleId=7...
60,월요일,모스크바의 여명,https://comic.naver.com/webtoon/list?titleId=7...
373,금요일,절대복종,https://comic.naver.com/webtoon/list?titleId=7...
278,목요일,트롤트랩,https://comic.naver.com/webtoon/list?titleId=7...


In [210]:
df.to_csv('output/네이버_요일별웹툰.csv', index=False, encoding='cp949')